# Imports

In [1]:
import matplotlib.patches as mpatches
from matplotlib.patches import Rectangle
import statsmodels as sm
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings('ignore')

# Data

In [2]:
data = pd.read_csv('./data/mlb-pitches.csv')
data.drop(columns = ['Unnamed: 0'], inplace = True) 

font_title = {
    'size': 12,
    'weight': 'bold',
    'verticalalignment': 'center_baseline',
    'horizontalalignment': 'center'
}

pd.set_option('max_columns', None)
print(data.shape)
data.head(3)

(708653, 76)


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,post_away_score,post_home_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bauer_units,is_strike,pitch_count,swing_miss,plate_-x,pfx_-x
0,FF,2021-10-03,92.3,1.40,6.80,"Smith, Will",596019,519293,out,hit_into_play,1.0,"Francisco Lindor grounds out, third baseman Eh...",R,R,L,ATL,NYM,X,5.0,ground_ball,1,2,2021,8.28,16.56,-0.69,2.83,NaN,NaN,NaN,2,9,Top,77.54,155.31,-6.833043,-134.166485,-7.361843,9.708393,26.562803,-14.083224,3.32,1.51,13.0,95.2,-13.0,92.8,2330.0,6.5,632254,54.03,0.174,0.158,0.0,1.0,0.0,0.0,2.0,61,4,4-Seam Fastball,5,0,0,5,Infield shift,Strategic,148.0,0.0,-0.073,25.243770,1,1-2,0,0.69,-8.28
1,SL,2021-10-03,80.6,1.60,6.64,"Smith, Will",596019,519293,NaN,foul,4.0,"Francisco Lindor grounds out, third baseman Eh...",R,R,L,ATL,NYM,S,NaN,NaN,1,1,2021,-9.24,5.76,-0.71,2.62,NaN,NaN,NaN,2,9,Top,NaN,NaN,-3.700232,-117.430885,-3.266842,-6.531123,19.793390,-27.369114,3.32,1.51,108.0,75.3,75.0,81.2,2254.0,6.4,632254,54.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,3,Slider,5,0,0,5,Infield shift,Strategic,315.0,0.0,-0.027,27.965261,1,1-1,0,0.71,9.24
2,CU,2021-10-03,75.5,1.46,6.88,"Smith, Will",596019,519293,NaN,foul,5.0,"Francisco Lindor grounds out, third baseman Eh...",R,R,L,ATL,NYM,S,NaN,NaN,1,0,2021,-7.80,-6.12,-0.04,2.46,NaN,NaN,NaN,2,9,Top,NaN,NaN,-1.977183,-109.901781,-1.155694,-4.872924,20.602334,-36.262184,3.32,1.51,157.0,83.5,65.0,75.2,1940.0,6.2,632254,54.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,2,Curveball,5,0,0,5,Infield shift,Standard,328.0,0.0,-0.020,25.695364,1,1-0,0,0.04,7.80


In [3]:
ff = data.loc[data['pitch_type'] == 'FF']
ff.describe()

relhieght = ff.loc[(ff['pfx_z'] > 14) & (ff['release_pos_z'] < 5.75)]
print(relhieght.shape)
relhieght.head()

(51749, 76)


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,post_away_score,post_home_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bauer_units,is_strike,pitch_count,swing_miss,plate_-x,pfx_-x
154,FF,2021-10-03,92.2,-2.07,5.33,"Williams, Trevor",624585,592866,NaN,called_strike,6.0,Jorge Soler walks.,R,R,R,ATL,NYM,S,NaN,NaN,0,0,2021,-3.84,14.16,0.80,2.55,NaN,NaN,NaN,1,3,Bot,NaN,NaN,8.061745,-134.011118,-3.848069,-5.507999,27.977422,-17.320602,3.93,1.77,NaN,NaN,NaN,93.1,2107.0,6.8,632254,53.71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,1,4-Seam Fastball,2,0,0,2,Infield shift,Standard,223.0,0.000,-0.027,22.852495,1,0-0,0,-0.80,3.84
173,FF,2021-10-03,95.9,-1.86,5.66,"Morton, Charlie",606299,450203,NaN,ball,12.0,Jose Peraza singles on a line drive to center ...,R,R,R,ATL,NYM,B,NaN,NaN,0,2,2021,-11.64,14.16,0.44,4.18,NaN,NaN,NaN,0,3,Top,NaN,NaN,8.281389,-139.463813,-0.947189,-14.515045,33.264954,-16.807547,3.41,1.58,NaN,NaN,NaN,95.7,2441.0,6.3,632254,54.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,3,4-Seam Fastball,2,0,0,2,Standard,Standard,235.0,0.000,0.022,25.453597,0,0-2,0,-0.44,11.64
188,FF,2021-10-03,92.1,-2.25,5.24,"Williams, Trevor",592696,592866,NaN,foul,3.0,Eddie Rosario strikes out swinging.,R,L,R,ATL,NYM,S,NaN,NaN,0,2,2021,-2.28,15.12,0.39,3.16,NaN,NaN,NaN,0,2,Bot,NaN,NaN,7.147059,-134.004730,-2.234491,-3.735811,26.892975,-16.598262,3.41,1.56,NaN,NaN,NaN,93.0,2297.0,6.7,632254,53.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,3,4-Seam Fastball,2,0,0,2,Infield shift,Standard,212.0,0.000,0.000,24.940282,1,0-2,0,-0.39,2.28
246,FF,2021-10-03,94.6,-2.88,5.60,"Wendelken, J.B.",606132,605525,out,hit_into_play,9.0,"Raimel Tapia grounds into a force out, first b...",R,L,R,ARI,COL,X,3.0,ground_ball,3,2,2021,-1.56,15.36,0.75,1.50,NaN,624513.0,453568.0,1,9,Top,151.59,172.53,9.733088,-137.152765,-7.797632,-4.031328,33.096026,-14.369356,3.49,1.60,4.0,71.9,-18.0,94.0,2484.0,6.3,632253,54.17,0.049,0.044,0.0,1.0,0.0,0.0,2.0,73,6,4-Seam Fastball,4,4,4,4,Standard,Standard,213.0,0.078,-0.680,26.257928,1,3-2,0,-0.75,1.56
254,FF,2021-10-03,94.6,-2.81,5.64,"Wendelken, J.B.",624513,605525,NaN,foul,6.0,Dom Nunez walks.,R,L,R,ARI,COL,S,NaN,NaN,3,2,2021,-3.12,14.52,0.50,2.81,NaN,NaN,NaN,1,9,Top,NaN,NaN,9.184697,-137.474217,-4.366835,-5.341805,31.296633,-16.042191,3.41,1.56,147.0,60.9,57.0,94.3,2407.0,6.2,632253,54.26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71,6,4-Seam Fastball,4,4,4,4,Infield shift,Standard,220.0,0.000,0.000,25.443975,1,3-2,0,-0.50,3.12
